In [1]:
import pandas as pd
df=pd.read_csv("C:/Users/Udith P Kidiyoor/Downloads/THE 2016-2026/THE World University Rankings 2016-2026.csv")
df.head()

,Rank,Name,Country,Student Population,Students to Staff Ratio,International Students,Female to Male Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Year
0,1,California Institute of Technology,United States,2243,6.9,26%,33 : 67,95.2,95.6,97.6,99.8,97.8,64.0,2016
1,2,University of Oxford,United Kingdom,19920,11.6,34%,46:54:00,94.2,86.5,98.9,98.8,73.1,94.4,2016
2,3,Stanford University,United States,15596,7.8,22%,42:58:00,93.9,92.5,96.2,99.9,63.3,76.3,2016
3,4,University of Cambridge,United Kingdom,18810,11.8,34%,46:54:00,92.8,88.2,96.7,97.0,55.0,91.5,2016
4,5,Massachusetts Institute of Technology,United States,11074,9.0,33%,37 : 63,92.0,89.4,88.6,99.7,95.4,84.0,2016


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16713 entries, 0 to 16712
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Rank                     16713 non-null  int64  
 1   Name                     16713 non-null  object 
 2   Country                  16713 non-null  object 
 3   Student Population       16713 non-null  object 
 4   Students to Staff Ratio  16713 non-null  float64
 5   International Students   16711 non-null  object 
 6   Female to Male Ratio     15953 non-null  object 
 7   Overall Score            16713 non-null  float64
 8   Teaching                 16713 non-null  float64
 9   Research Environment     16713 non-null  float64
 10  Research Quality         16713 non-null  float64
 11  Industry Impact          16713 non-null  float64
 12  International Outlook    16713 non-null  float64
 13  Year                     16713 non-null  int64  
dtypes: float64(7), int64(2

In [3]:
df.isna().sum()

Rank                         0
Name                         0
Country                      0
Student Population           0
Students to Staff Ratio      0
International Students       2
Female to Male Ratio       760
Overall Score                0
Teaching                     0
Research Environment         0
Research Quality             0
Industry Impact              0
International Outlook        0
Year                         0
dtype: int64

In [4]:
df["International Students"] = (
    df["International Students"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.strip()
)

df["International Students"] = pd.to_numeric(
    df["International Students"], errors="coerce"
)


In [5]:
df["Student Population"] = (
    df["Student Population"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
)

df["Student Population"] = pd.to_numeric(
    df["Student Population"], errors="coerce"
)


In [6]:
df["Students to Staff Ratio"] = pd.to_numeric(
    df["Students to Staff Ratio"], errors="coerce"
)


In [7]:
df[[
    "Student Population",
    "Students to Staff Ratio",
    "International Students"
]].dtypes


Student Population           int64
Students to Staff Ratio    float64
International Students     float64
dtype: object

In [8]:
df_2025 = df[df["Year"] == 2025].copy()


In [9]:
manipal = df_2025[
    df_2025["Name"].str.contains("Manipal", case=False, na=False)
].iloc[0]

manipal


Rank                                                       826
Name                       Manipal Academy of Higher Education
Country                                                  India
Student Population                                       23947
Students to Staff Ratio                                    8.6
International Students                                    12.0
Female to Male Ratio                                  47:53:00
Overall Score                                           37.666
Teaching                                                  42.6
Research Environment                                      17.0
Research Quality                                          47.8
Industry Impact                                           50.1
International Outlook                                     51.0
Year                                                      2025
Name: 13255, dtype: object

In [10]:
manipal_vals = {
    "Student Population": float(manipal["Student Population"]),
    "Students to Staff Ratio": float(manipal["Students to Staff Ratio"]),
    "International Students": float(manipal["International Students"]),
}

manipal_rank = int(manipal["Rank"])

manipal_vals, manipal_rank


({'Student Population': 23947.0,
  'Students to Staff Ratio': 8.6,
  'International Students': 12.0},
 826)

In [11]:
candidates = df_2025[
    (df_2025["Rank"] < manipal_rank) &
    (df_2025["Rank"]>=200) & (df_2025["Rank"] <= 600)
].copy()


In [12]:
POP_TOL = 0.30      # ±30%
SSR_TOL = 0.25      # ±25%
INTL_TOL = 0.25     # ±25%


In [13]:
candidates["pop_diff"] = abs(
    candidates["Student Population"] - manipal_vals["Student Population"]
) / manipal_vals["Student Population"]

candidates["ssr_diff"] = abs(
    candidates["Students to Staff Ratio"] - manipal_vals["Students to Staff Ratio"]
) / manipal_vals["Students to Staff Ratio"]

candidates["intl_diff"] = abs(
    candidates["International Students"] - manipal_vals["International Students"]
) / manipal_vals["International Students"]


In [14]:
candidates["similarity_count"] = (
    (candidates["pop_diff"] <= POP_TOL).astype(int) +
    (candidates["ssr_diff"] <= SSR_TOL).astype(int) +
    (candidates["intl_diff"] <= INTL_TOL).astype(int)
)

similar = candidates[candidates["similarity_count"] >= 2].copy()


In [15]:
final = similar.sort_values(
    ["similarity_count", "Rank"],
    ascending=[False, True]
)

final[[
    "Name",
    "Country",
    "Rank",
    "Student Population",
    "Students to Staff Ratio",
    "International Students",
    "similarity_count"
]].head(25)


,Name,Country,Rank,Student Population,Students to Staff Ratio,International Students,similarity_count
12642,Tianjin University,China,213,27081,8.9,15.0,3
12695,University of Southern Denmark,Denmark,266,17572,9.5,14.0,3
12736,Kyushu University,Japan,307,18342,7.8,13.0,3
12805,Hokkaido University,Japan,376,17687,8.2,10.0,3
12631,University of Erlangen-Nuremberg,Germany,202,28219,33.9,15.0,2
12637,Goethe University Frankfurt,Germany,208,27937,20.1,13.0,2
12643,Georgetown University,United States,214,17579,9.8,18.0,2
12644,University of Gothenburg,Sweden,215,21568,11.5,13.0,2
12661,University of Miami,United States,232,18354,10.8,13.0,2
12683,Aalborg University,Denmark,254,16871,14.4,13.0,2


In [16]:
final_200_500 = final[
    (final["Rank"] >= 200) &
    (final["Rank"] <= 500)
].copy()
final_200_500

,Rank,Name,Country,Student Population,Students to Staff Ratio,International Students,Female to Male Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Year,pop_diff,ssr_diff,intl_diff,similarity_count
12642,213,Tianjin University,China,27081,8.9,15.0,NaN,57.9405,46.8,47.7,73.4,99.6,57.3,2025,0.130872,0.034884,0.250000,3
12695,266,University of Southern Denmark,Denmark,17572,9.5,14.0,53:47:00,55.1385,30.3,35.0,88.3,95.0,76.8,2025,0.266213,0.104651,0.166667,3
12736,307,Kyushu University,Japan,18342,7.8,13.0,29 : 71,53.2950,53.8,47.6,53.3,98.5,49.2,2025,0.234059,0.093023,0.083333,3
12805,376,Hokkaido University,Japan,17687,8.2,10.0,29 : 71,50.2110,52.6,44.7,47.8,95.9,47.4,2025,0.261411,0.046512,0.166667,3
12631,202,University of Erlangen-Nuremberg,Germany,28219,33.9,15.0,51:49:00,58.6405,42.1,48.4,78.5,100.0,61.8,2025,0.178394,2.941860,0.250000,2
12637,208,Goethe University Frankfurt,Germany,27937,20.1,13.0,59:41:00,58.3530,42.5,43.3,82.5,83.9,68.7,2025,0.166618,1.337209,0.083333,2
12643,214,Georgetown University,United States,17579,9.8,18.0,55:45:00,57.8055,50.4,34.7,83.7,92.3,54.3,2025,0.265921,0.139535,0.500000,2
12644,215,University of Gothenburg,Sweden,21568,11.5,13.0,NaN,57.7750,34.6,43.4,90.2,77.3,64.4,2025,0.099344,0.337209,0.083333,2
12661,232,University of Miami,United States,18354,10.8,13.0,54:46:00,57.0055,46.5,34.8,83.6,79.9,65.6,2025,0.233557,0.255814,0.083333,2
12683,254,Aalborg University,Denmark,16871,14.4,13.0,NaN,55.6010,31.7,44.7,80.9,87.6,73.5,2025,0.295486,0.674419,0.083333,2


In [17]:
df_metrics = pd.read_csv("C:/Users/Udith P Kidiyoor/Downloads/2026-01-09_wur_metrics_2025/metric values.csv")
df_metrics.head()

,Unnamed: 0,Doctorate bachelor ratio,Doctorate staff ratio,Institutional income,Student staff ratio,Teaching reputation,Research productivity,Research income,Research reputation,Citation impact,Research excellence,Research strength,Research influence,Industry income,Patents,International co-authorship,International staff,International students,Studying abroad
0,Manipal Academy of Higher Education,0.27,0.49,356971,8.6,68.5,4.9,39008,61.4,1.16,24.40,1.05,79.66,22819,36.56,30%,7.7%,12%,0.02
1,RMIT University,0.05,0.28,764190,23.8,109.7,11.9,88151,154.3,1.58,86.30,1.86,96.52,21357,103.54,57%,64.3%,34.9%,0.07
2,National University of Singapore,0.09,0.44,2817421,21.0,2592.5,16.1,734555,3603.8,2.05,206.31,2.08,99.50,129617,611.88,67.6%,57.9%,26.2%,0.04
3,University of Connecticut,0.14,0.30,1307135,11.6,121.6,7.7,111750,177.6,1.45,62.59,1.59,93.04,4954,321.27,35.1%,22.5%,11.3%,0
4,Justus Liebig University Giessen,0.12,0.61,1027969,38.7,53.8,15.3,244195,95.2,1.43,65.03,1.53,88.05,14537,278.65,53.5%,9.6%,11.5%,NaN


In [18]:
df_metrics = df_metrics.reset_index()
df_metrics.head()

,index,Unnamed: 0,Doctorate bachelor ratio,Doctorate staff ratio,Institutional income,Student staff ratio,Teaching reputation,Research productivity,Research income,Research reputation,Citation impact,Research excellence,Research strength,Research influence,Industry income,Patents,International co-authorship,International staff,International students,Studying abroad
0,0,Manipal Academy of Higher Education,0.27,0.49,356971,8.6,68.5,4.9,39008,61.4,1.16,24.40,1.05,79.66,22819,36.56,30%,7.7%,12%,0.02
1,1,RMIT University,0.05,0.28,764190,23.8,109.7,11.9,88151,154.3,1.58,86.30,1.86,96.52,21357,103.54,57%,64.3%,34.9%,0.07
2,2,National University of Singapore,0.09,0.44,2817421,21.0,2592.5,16.1,734555,3603.8,2.05,206.31,2.08,99.50,129617,611.88,67.6%,57.9%,26.2%,0.04
3,3,University of Connecticut,0.14,0.30,1307135,11.6,121.6,7.7,111750,177.6,1.45,62.59,1.59,93.04,4954,321.27,35.1%,22.5%,11.3%,0
4,4,Justus Liebig University Giessen,0.12,0.61,1027969,38.7,53.8,15.3,244195,95.2,1.43,65.03,1.53,88.05,14537,278.65,53.5%,9.6%,11.5%,NaN


In [19]:
df_metrics.columns

Index(['index', 'Unnamed: 0', 'Doctorate bachelor ratio',
       'Doctorate staff ratio', 'Institutional income', 'Student staff ratio',
       'Teaching reputation', 'Research productivity', 'Research income',
       'Research reputation', 'Citation impact', 'Research excellence',
       'Research strength', 'Research influence', 'Industry income', 'Patents',
       'International co-authorship', 'International staff',
       'International students', 'Studying abroad'],
      dtype='object')

In [20]:
df_metrics = df_metrics.rename(columns={
    "Unnamed: 0": "Name"
})


In [21]:
df_metrics = df_metrics.drop(columns=["level_0", "index"], errors="ignore")
df_metrics.head()

,Name,Doctorate bachelor ratio,Doctorate staff ratio,Institutional income,Student staff ratio,Teaching reputation,Research productivity,Research income,Research reputation,Citation impact,Research excellence,Research strength,Research influence,Industry income,Patents,International co-authorship,International staff,International students,Studying abroad
0,Manipal Academy of Higher Education,0.27,0.49,356971,8.6,68.5,4.9,39008,61.4,1.16,24.40,1.05,79.66,22819,36.56,30%,7.7%,12%,0.02
1,RMIT University,0.05,0.28,764190,23.8,109.7,11.9,88151,154.3,1.58,86.30,1.86,96.52,21357,103.54,57%,64.3%,34.9%,0.07
2,National University of Singapore,0.09,0.44,2817421,21.0,2592.5,16.1,734555,3603.8,2.05,206.31,2.08,99.50,129617,611.88,67.6%,57.9%,26.2%,0.04
3,University of Connecticut,0.14,0.30,1307135,11.6,121.6,7.7,111750,177.6,1.45,62.59,1.59,93.04,4954,321.27,35.1%,22.5%,11.3%,0
4,Justus Liebig University Giessen,0.12,0.61,1027969,38.7,53.8,15.3,244195,95.2,1.43,65.03,1.53,88.05,14537,278.65,53.5%,9.6%,11.5%,NaN


In [22]:
df_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Name                         54 non-null     object 
 1   Doctorate bachelor ratio     54 non-null     float64
 2   Doctorate staff ratio        54 non-null     float64
 3   Institutional income         54 non-null     int64  
 4   Student staff ratio          54 non-null     float64
 5   Teaching reputation          54 non-null     float64
 6   Research productivity        54 non-null     float64
 7   Research income              54 non-null     int64  
 8   Research reputation          54 non-null     float64
 9   Citation impact              54 non-null     float64
 10  Research excellence          54 non-null     float64
 11  Research strength            54 non-null     float64
 12  Research influence           54 non-null     float64
 13  Industry income       

In [23]:
def clean_name(s):
    return (
        s.astype(str)
         .str.lower()
         .str.strip()
    )

final_200_500["name_clean"] = clean_name(final_200_500["Name"])
df_metrics["name_clean"] = clean_name(df_metrics["Name"])


In [24]:
matches = final_200_500[
    final_200_500["name_clean"].isin(df_metrics["name_clean"])
]

len(matches)


6

In [25]:
matches[[
    "Name",
    "Country",
    "Rank",
    "similarity_count"
]]


,Name,Country,Rank,similarity_count
12642,Tianjin University,China,213,3
12723,University of Malaya,Malaysia,294,2
12766,University of Connecticut,United States,337,2
12782,Justus Liebig University Giessen,Germany,353,2
12820,University of Delaware,United States,391,2
12854,Universiti Kebangsaan Malaysia,Malaysia,425,2


In [26]:
similarity_vars = {
    "Doctorate bachelor ratio": 0.30,
    "Doctorate staff ratio": 0.30,
    "Student staff ratio": 0.25,
    "Research productivity": 0.30,
    "Research income": 0.35,
    "Citation impact": 0.30,
    "Industry income": 0.35,
    "Patents": 0.40,
    "International students": 0.25,
    "International staff": 0.25,
    "International co-authorship": 0.25
}


In [31]:
manipal_metrics = df_metrics[
    df_metrics["Name"].str.contains("Manipal", case=False, na=False)
].iloc[0]


In [32]:
candidates = df_metrics.copy()


In [33]:
similarity_vars.keys()


dict_keys(['Doctorate bachelor ratio', 'Doctorate staff ratio', 'Student staff ratio', 'Research productivity', 'Research income', 'Citation impact', 'Industry income', 'Patents', 'International students', 'International staff', 'International co-authorship'])

In [34]:
for col in similarity_vars.keys():
    df_metrics[col] = (
        df_metrics[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.strip()
    )
    df_metrics[col] = pd.to_numeric(df_metrics[col], errors="coerce")


In [35]:
df_metrics[list(similarity_vars.keys())].dtypes


Doctorate bachelor ratio       float64
Doctorate staff ratio          float64
Student staff ratio            float64
Research productivity          float64
Research income                  int64
Citation impact                float64
Industry income                  int64
Patents                        float64
International students         float64
International staff            float64
International co-authorship    float64
dtype: object

In [36]:
manipal_metrics = df_metrics[
    df_metrics["Name"].str.contains("Manipal", case=False, na=False)
].iloc[0]


In [37]:
type(manipal_metrics["Research income"])


numpy.int64

In [38]:
candidates = df_metrics.copy()


In [39]:
def similarity_count(row, ref, vars_tol):
    count = 0
    for var, tol in vars_tol.items():
        if pd.notna(row[var]) and pd.notna(ref[var]) and ref[var] != 0:
            diff = abs(row[var] - ref[var]) / abs(ref[var])
            if diff <= tol:
                count += 1
    return count

candidates["similarity_count"] = candidates.apply(
    similarity_count,
    axis=1,
    ref=manipal_metrics,
    vars_tol=similarity_vars
)


In [40]:
similar = candidates[candidates["similarity_count"] >= 3].copy()

similar = similar.sort_values(
    "similarity_count",
    ascending=False
)

top_20 = similar.head(20)
top_20

,Name,Doctorate bachelor ratio,Doctorate staff ratio,Institutional income,Student staff ratio,Teaching reputation,Research productivity,Research income,Research reputation,Citation impact,...,Research strength,Research influence,Industry income,Patents,International co-authorship,International staff,International students,Studying abroad,name_clean,similarity_count
0,Manipal Academy of Higher Education,0.27,0.49,356971,8.6,68.5,4.9,39008,61.4,1.16,...,1.05,79.66,22819,36.56,NaN,NaN,NaN,0.02,manipal academy of higher education,8
5,Banaras Hindu University,0.19,0.84,692180,8.9,105.9,6.4,29170,88.6,1.43,...,1.41,66.31,100,29.63,NaN,NaN,NaN,0,banaras hindu university,5
9,Universiti Kebangsaan Malaysia,0.20,0.45,586933,10.4,103.4,6.7,82717,145.1,1.27,...,1.36,85.91,6907,41.20,NaN,NaN,NaN,0.02,universiti kebangsaan malaysia,5
11,Jamia Millia Islamia,0.26,0.49,332945,11.7,51.9,4.2,49941,34.5,1.59,...,1.78,62.54,11314,11.10,NaN,NaN,NaN,0.01,jamia millia islamia,4
26,JSS Academy of Higher Education and Research,0.29,0.60,183160,9.9,7.0,2.5,12726,7.9,1.40,...,0.96,47.89,7179,7.66,NaN,NaN,NaN,0.01,jss academy of higher education and research,4
8,Indian Institute of Technology Ropar,0.21,0.38,779602,15.4,32.1,12.0,56942,43.3,1.11,...,1.44,63.28,14783,13.40,NaN,NaN,NaN,0,indian institute of technology ropar,3
6,SIMATS - Saveetha Institute of Medical And Tec...,0.19,0.33,320818,10.0,9.6,7.7,29316,13.9,2.09,...,1.93,56.87,3920,2.12,NaN,NaN,NaN,0.02,simats - saveetha institute of medical and tec...,3
7,Kalinga Institute of Industrial Technology (KIIT),0.19,0.38,342895,12.2,33.5,3.5,100940,25.9,1.51,...,1.39,59.69,73804,6.79,NaN,NaN,NaN,0.02,kalinga institute of industrial technology (kiit),3
19,Universiti Sains Malaysia,0.23,0.42,478832,12.3,95.4,10.5,24401,141.6,1.25,...,1.30,94.41,12231,57.39,NaN,NaN,NaN,0.02,universiti sains malaysia,3
21,Indian Institute of Science,2.91,0.60,1311188,9.8,421.1,19.2,316167,663.0,1.07,...,1.28,96.31,115833,184.72,NaN,NaN,NaN,0,indian institute of science,3
